In [1]:

class Bisect:
    """
    Класс, реализующий поиск корней уравнения f(x)=0:
    1) Метод бисекции для одного корня (find_one_koran).
    2) Сканирование всего интервала, чтобы найти все корни
    (find_all_korans).
    """
    def __init__(self, f, epsilon=1e-7, max_iter=100, merge_epsilon=1e-4):
        """
        Инициализация класса
        Параметры:
        F: callable
        Функция: f(x)
        Epsilon: float
        Точность (для критерия остановки по длине отрезка в
        бисекции)
        Max_iter: int
        Максимальное кол-во итераций в методе бисекции
        Merge_epsilon: float
        Порог слияния близких корней при поиске всех корней
        """
        self.f = f
        self.epsilon = epsilon
        self.max_iter = max_iter
        self.merge_epsilon = merge_epsilon

    def find_one_koran(self, a, b):
        """
        Ищет ОДИН корень f(x)=0 на отрезке [a,b] методом бисекции.
        Предполагает, что f(a)*f(b) <= 0, иначе выбрасывает ValueError.
        Возвращает:
        koran: float-найденныйкорень
        steps:list[dict]- данные окаждомшаге (a_k,b_k,mid,
        f(mid)).
        """
        fa = self.f(a)
        fb = self.f(b)
        if fa * fb > 0:
            raise ValueError(
                'Нетcмены знака наотрезке[a,b]: f(a)*f(b) >0=> корень негарантирован'
            )

        steps = []
        for k in range(self.max_iter):
            mid = 0.5 * (a + b)
            fmid = self.f(mid)
            steps.append({
                'iter': k,
                'a': a,
                'b': b,
                'mid': mid,
                'f(mid)': fmid
            })

            if abs(b - a) < self.epsilon:
                break

            if fa * fmid <= 0:
                #корень в[a,mid]
                b = mid
                fb = fmid
            else:
                #корень в[mid,b]
                a = mid
                fa = fmid

        koran = 0.5 * (a + b)
        return koran, steps

    def find_all_korans(self, A, B, step=0.1):
        """
        ИщетВСЕ корни f(x)=0на[A,B] через "сканирование +бисекцию":
        1)Делим[A,B]на интервалы длиныstep.
        2)Накаждом[x_i, x_{i+1}] проверяем f(x_i)*f(x_{i+1}) <=0.
        3)Еслида, вызываем find_one_koran()=> получаем одинкорень.
        4)Сливаемпочтиодинаковыекорни (расстояние <
        self.merge_epsilon).
        Возвращает:
        korans:list[float]- список отсорт. уник.корней.
        """
        # Создаем список точек с заданным шагом
        x_points = []
        current = A
        while current <= B:
            x_points.append(current)
            current += step
        
        # Добавляем B, если он еще не добавлен
        if x_points[-1] < B:
            x_points.append(B)

        found_korans = []
        for i in range(len(x_points) - 1):
            x_left = x_points[i]
            x_right = x_points[i + 1]
            f_left = self.f(x_left)
            f_right = self.f(x_right)

            if f_left * f_right > 0:
                continue

            try:
                koran, yoooo = self.find_one_koran(x_left, x_right)
                found_korans.append(koran)
            except ValueError:
                #Корень ровно награнице
                if abs(f_left) < self.epsilon:
                    found_korans.append(x_left)
                elif abs(f_right) < self.epsilon:
                    found_korans.append(x_right)

        #Слияние почти одинаковых корней
        found_korans.sort()
        merged_korans = []
        for r in found_korans:
            if not merged_korans:
                merged_korans.append(r)
            else:
                if abs(r - merged_korans[-1]) < self.merge_epsilon:
                    merged_korans[-1] = 0.5 * (merged_korans[-1] + r)
                else:
                    merged_korans.append(r)

        return merged_korans


In [4]:


import copy
from matplotlib.figure import Figure
import matplotlib.pyplot as plt
import math
import random
# Assume Bisect class is defined elsewhere and available globally for find_eigenvalues
# Example:
# class Bisect:
#    def __init__(self, func, epsilon, max_iter, merge_epsilon): pass
#    def find_all_korans(self, start, end, step): return [1.0, 2.0] # Placeholder

class Matrix:
    round_precision = 3
    round_precision_eig = 2

    def __init__(self, data):
        if not isinstance(data, list) or not all(isinstance(row, list) for row in data):
            raise TypeError("Input must be a list of lists")
        # Basic check for consistent row lengths, assumes non-empty matrix if applicable
        if data and data[0]:
             cols = len(data[0])
             if not all(len(row) == cols for row in data):
                 raise ValueError("All rows must have the same number of columns")
        self.data = data
        self.rows = len(data)
        self.cols = len(data[0]) if self.rows > 0 else 0

    def dot(self, other): #like
        if not isinstance(other, Matrix):
             raise TypeError("Can only multiply by another Matrix object")
        if self.cols != other.rows:
            raise ValueError("Matrix dimensions incompatible for multiplication")

        result_data = [[0 for _ in range(other.cols)] for _ in range(self.rows)]
        for i in range(self.rows):
            for j in range(other.cols):
                sum_val = 0
                for k in range(self.cols):
                    sum_val += self.data[i][k] * other.data[k][j]
                result_data[i][j] = sum_val
        return Matrix(result_data)

    def sumM(self, other): #like
        if not isinstance(other, Matrix):
            raise TypeError("Can only add another Matrix object")
        if self.rows != other.rows or self.cols != other.cols:
            raise ValueError("Matrices must be the same size for addition")

        result_data = [[0 for _ in range(self.cols)] for _ in range(self.rows)]
        for i in range(self.rows):
            for j in range(self.cols):
                result_data[i][j] = round(self.data[i][j] + other.data[i][j], self.round_precision)
        return Matrix(result_data)

    def tr(self): #лайк
        result_data = [[self.data[j][i] for j in range(self.rows)] for i in range(self.cols)]
        return Matrix(result_data)

    def scalar(self, scalar_value):#like
        if not isinstance(scalar_value, (int, float)):
            raise TypeError("Scalar value must be a number")
        result_data = [[round(scalar_value * self.data[i][j], self.round_precision) for j in range(self.cols)] for i in range(self.rows)]
        return Matrix(result_data)

    def determinant(self):
        if self.rows != self.cols:
            raise ValueError("Matrix must be square to calculate determinant")
        n = self.rows
        matrix_data = self.data

        if n == 1:
            return matrix_data[0][0]
        if n == 2:
            return matrix_data[0][0] * matrix_data[1][1] - matrix_data[0][1] * matrix_data[1][0]

        det = 0
        for j in range(n):
            submatrix_data = [row[:j] + row[j+1:] for row in matrix_data[1:]]
            submatrix = Matrix(submatrix_data)
            sign = (-1) ** j
            det += sign * matrix_data[0][j] * submatrix.determinant()
        return det

    def trace(self): #like
        if self.rows != self.cols:
            raise ValueError("Matrix must be square to calculate trace.")
        if self.rows == 0:
            return 0
        return sum(self.data[i][i] for i in range(self.rows))

    
    def identity(n): #like
        data = [[1 if i == j else 0 for j in range(n)] for i in range(n)]
        return Matrix(data)

    def characteristic_polynomial(self): #like
        if self.rows != self.cols:
            raise ValueError("Matrix must be square.")
        n = self.rows
        if n == 0:
            raise ValueError("Matrix is empty.")

        I = Matrix.identity(n)
        B = Matrix(copy.deepcopy(I.data))
        coefficients = [1.0]
        A_matrix = self

        for k in range(1, n + 1):
            M = A_matrix.dot(B)
            tr_M = M.trace()
            p_k = tr_M / k if k != 0 else 0
            coefficients.append(-p_k)
            scaled_I = I.scalar(p_k)
            B = M.sumM(scaled_I.scalar(-1.0))

        return coefficients

    
    def _polynomial_derivative(coeffs): #like
        derivative = {}
        for power, coeff in coeffs.items():
            if power > 0:
                derivative[power - 1] = power * coeff
        return derivative

    
    def _create_polynomial_function(coeffs): #лайк
        def poly_func(x):
            return sum(coeff * (x ** power) for power, coeff in coeffs.items())
        return poly_func

    def find_eigenvalues(self, epsilon=1e-6, max_iter=1000, step=0.05,
                         search_range=(-10000, 10000)): #лайк
        if self.rows != self.cols:
            raise ValueError("Matrix must be square.")
        n = self.rows
        if n == 0: return [], []

        coefs_list = self.characteristic_polynomial()
        coefs_dict = dict(zip(range(len(coefs_list)-1, -1, -1), coefs_list))

        f = Matrix._create_polynomial_function(coefs_dict)

        derivatives = {}
        current_coeffs = coefs_dict.copy()
        for i in range(1, len(coefs_list)):
            current_coeffs = Matrix._polynomial_derivative(current_coeffs)
            if not current_coeffs: break
            derivatives[i] = Matrix._create_polynomial_function(current_coeffs)

        try:
            # This external dependency needs to be provided in the environment
            zist = Bisect(f, epsilon=epsilon, max_iter=max_iter, merge_epsilon=epsilon)
            eigenvalues_raw = zist.find_all_korans(search_range[0], search_range[1], step)
        except NameError:
             print("Warning: Bisect class not found. Eigenvalue calculation requires Bisect.")
             return [], [] # Cannot proceed without Bisect

        eigenvalues_rounded = [round(e, self.round_precision_eig) for e in eigenvalues_raw]

        def get_algk(eigenvalue):
            if abs(f(eigenvalue)) >= 1e-3: return 0
            mult = 1
            for i in range(1, len(coefs_list)):
                if i in derivatives:
                    deriv_func = derivatives[i]
                    if abs(deriv_func(eigenvalue)) < 1e-3: mult += 1
                    else: break
                else: break
            return mult

        unique_eigenvalues = sorted(list(set(eigenvalues_rounded)))
        algebraic_algks = [get_algk(ev) for ev in unique_eigenvalues]

        final_eigenvalues = []
        final_algks = []
        total_multiplicity = 0
        for ev, mult in zip(unique_eigenvalues, algebraic_algks):
             if mult > 0:
                 final_eigenvalues.append(ev)
                 final_algks.append(mult)
                 total_multiplicity += mult

        # Check if total multiplicity matches matrix dimension
        if total_multiplicity != n:
            print(f"Warning: Sum of algebraic multiplicities ({total_multiplicity}) does not match matrix dimension ({n}). Results may be incomplete or inaccurate.")


        print("Eigenvalues and their Algebraic Multiplicities:")
        if not final_eigenvalues:
             print(" No eigenvalues found.")
        for i, (ev, mult) in enumerate(zip(final_eigenvalues, final_algks), 1):
            print(f'λ_{i} = {ev}, a(λ_{i}) = {mult}')

        return final_eigenvalues, final_algks

    
    def find_eigenvalues_and_vectors(self, max_iter=10000, tol=1e-5):

        if self.rows != self.cols:
            raise ValueError("Matrix must be square")
    
        A = Matrix(copy.deepcopy(self.data))
        n = self.rows
        eigenvectors = Matrix.identity(n)
    
        for _ in range(max_iter):
            # QR-разложение
            Q, R = Matrix.qr_decomposition(A)
            # Обновление A и накопление собственных векторов
            A = R.dot(Q)
            eigenvectors = eigenvectors.dot(Q)
        
            # Проверка сходимости (диагональная форма)
            off_diag_norm = 0.0
            for i in range(n):
                for j in range(n):
                    if i != j:
                        off_diag_norm += A.data[i][j]**2
            if off_diag_norm < tol:
                break
    
        # Извлечение собственных значений (диагональ A)
        eigenvalues = [A.data[i][i] for i in range(n)]
    
        # Нормализация собственных векторов
        normalized_vectors = []
        for i in range(n):
            vec = [eigenvectors.data[j][i] for j in range(n)]
            norm = math.sqrt(sum(x**2 for x in vec))
            if norm > tol:
                normalized_vectors.append([x/norm for x in vec])
            else:
                normalized_vectors.append(vec)
    
        return eigenvalues, normalized_vectors

    def qr_decomposition(self):
        n = self.rows
        Q = Matrix([[0.0]*n for _ in range(n)])
        R = Matrix([[0.0]*n for _ in range(n)])
    
        for j in range(n):
            # Вектор-столбец A
            v = [self.data[i][j] for i in range(n)]
        
            # Ортогонализация относительно предыдущих столбцов
            for i in range(j):
                R.data[i][j] = sum(Q.data[k][i] * self.data[k][j] for k in range(n))
                v = [v[k] - R.data[i][j] * Q.data[k][i] for k in range(n)]
        
            # Нормализация
            norm = math.sqrt(sum(x**2 for x in v))
            if norm < 1e-10:
                raise ValueError("Matrix is rank deficient")
            
            R.data[j][j] = norm
            for i in range(n):
                Q.data[i][j] = v[i] / norm
    
        return Q, R
    
    def solve(A_matrix, b_vector, max_solves=None): #like
        if not isinstance(A_matrix, Matrix): raise TypeError("A must be a Matrix object")
        if not isinstance(b_vector, list): raise TypeError("b must be a list")

        A = copy.deepcopy(A_matrix.data)
        b = copy.deepcopy(b_vector)
        n = A_matrix.rows
        m = A_matrix.cols

        if n == 0: return []
        if len(b) != n: raise ValueError("Matrix rows must match vector length")
        if m < n: print("Warning: Underdetermined system (more equations than variables)") # Though typically m >= n

        if max_solves is None: max_solves = m

        augmented = [A[i] + [b[i]] for i in range(n)]
        
        current_rows = n
        current_cols = m # Number of variables

        # Gaussian elimination with partial pivoting
        pivot_cols = []
        rank = 0
        pivot_row_map = list(range(n)) # Maps original row index to current position

        for j in range(current_cols): # Iterate through variable columns
             if rank >= current_rows: break # Stop if all rows processed

             # Find pivot row below current rank
             max_row_idx_local = rank
             for i in range(rank + 1, current_rows):
                  if abs(augmented[pivot_row_map[i]][j]) > abs(augmented[pivot_row_map[max_row_idx_local]][j]):
                      max_row_idx_local = i

             # Swap rows in the map
             pivot_row_map[rank], pivot_row_map[max_row_idx_local] = pivot_row_map[max_row_idx_local], pivot_row_map[rank]
             
             pivot_row_original_idx = pivot_row_map[rank]

             # Check if pivot is non-zero
             if abs(augmented[pivot_row_original_idx][j]) < 1e-10:
                  continue # Skip column if pivot is zero

             pivot_val = augmented[pivot_row_original_idx][j]
             pivot_cols.append(j) # Store pivot column index

             # Eliminate elements below the pivot in the current column j
             for i in range(rank + 1, current_rows):
                  target_row_original_idx = pivot_row_map[i]
                  factor = augmented[target_row_original_idx][j] / pivot_val
                  # Apply row operation R_i = R_i - factor * R_pivot
                  for k in range(j, current_cols + 1): # Include augmented column
                      augmented[target_row_original_idx][k] -= factor * augmented[pivot_row_original_idx][k]
                  # Zero out small numbers resulting from subtraction
                  if abs(augmented[target_row_original_idx][j]) < 1e-12:
                      augmented[target_row_original_idx][j] = 0.0
             
             rank += 1
        
        # Reorder rows physically based on final pivot_row_map for back-substitution
        ordered_matrix_data = [augmented[pivot_row_map[i]] for i in range(current_rows)]

        # Check for inconsistency
        for i in range(rank, current_rows):
             # Check rows that should be all zero in the coefficient part
             if all(abs(ordered_matrix_data[i][k]) < 1e-10 for k in range(current_cols)):
                  if abs(ordered_matrix_data[i][current_cols]) > 1e-10:
                       return None # Inconsistent system: 0 = non-zero

        # Back-substitution / Solution finding
        solutions = []
        free_variable_indices = [j for j in range(current_cols) if j not in pivot_cols]
        
        # Generate solutions
        # We need to find a particular solution and optionally basis vectors for null space if infinite solutions
        # The original recursive approach was complex; implementing a standard approach:
        
        if len(free_variable_indices) == 0: # Unique solution or possibly inconsistent (already checked)
            if rank < current_cols:
                 # This case (rank < cols but no free vars) implies dependent columns,
                 # which should have resulted in free vars. Might indicate issue.
                 # Or it could be an overdetermined system with a unique solution.
                 print("Warning: System rank might be less than number of variables, but no free variables identified.")
                 # Proceed assuming a unique solution is determined by the rank rows

            sol = [0.0] * current_cols
            for i in range(rank - 1, -1, -1):
                 pivot_col_idx = pivot_cols[i] # The column index for the pivot in this row
                 pivot_val = ordered_matrix_data[i][pivot_col_idx]
                 
                 if abs(pivot_val) < 1e-10: continue # Should not happen with proper pivoting

                 rhs = ordered_matrix_data[i][current_cols] # b value
                 for j in range(pivot_col_idx + 1, current_cols):
                      rhs -= ordered_matrix_data[i][j] * sol[j]
                 
                 sol[pivot_col_idx] = rhs / pivot_val
            
            solutions.append([round(val, Matrix.round_precision) for val in sol])

        else: # Infinite solutions
             # Find one particular solution (set free variables to 0)
             particular_sol = [0.0] * current_cols
             for i in range(rank - 1, -1, -1):
                  pivot_col_idx = pivot_cols[i]
                  pivot_val = ordered_matrix_data[i][pivot_col_idx]
                  if abs(pivot_val) < 1e-10: continue

                  rhs = ordered_matrix_data[i][current_cols]
                  for j in range(pivot_col_idx + 1, current_cols):
                       # Only subtract terms involving basic variables (already computed)
                       # or free variables (which are 0 for particular solution)
                       if j not in free_variable_indices:
                            rhs -= ordered_matrix_data[i][j] * particular_sol[j]
                       # else: term is ordered_matrix[i][j] * 0 = 0
                  
                  particular_sol[pivot_col_idx] = rhs / pivot_val
             
             # For finding multiple solutions as requested by max_solves:
             # We can generate them by assigning simple values (like 0, 1) to free variables.
             # Generate combinations of free variable assignments.
             num_free = len(free_variable_indices)
             if max_solves > 2**num_free: max_solves = 2**num_free # Limit to possible combinations
             
             count = 0
             # Iterate through assignments like (0,0,..), (1,0,..), (0,1,..), (1,1,..) etc.
             # Using binary representation of numbers 0 to max_solves-1 (or 2**num_free - 1)
             
             # Ensure max_solves is reasonable
             max_attempts = min(max_solves, 2**num_free if num_free < 10 else 1024) # Limit attempts for many free vars

             for i_assign in range(max_attempts):
                  if len(solutions) >= max_solves: break
                  
                  current_sol = [0.0] * current_cols
                  temp_assign = i_assign
                  
                  # Set free variable values for this iteration
                  free_var_values = {}
                  for k in range(num_free):
                      free_idx = free_variable_indices[k]
                      val = float(temp_assign % 2) # Assign 0 or 1
                      current_sol[free_idx] = val
                      free_var_values[free_idx] = val
                      temp_assign //= 2

                  # Calculate basic variable values based on these free var assignments
                  for i_row in range(rank - 1, -1, -1):
                       pivot_col_idx = pivot_cols[i_row]
                       pivot_val = ordered_matrix_data[i_row][pivot_col_idx]
                       if abs(pivot_val) < 1e-10: continue

                       rhs = ordered_matrix_data[i_row][current_cols]
                       for j_col in range(pivot_col_idx + 1, current_cols):
                            rhs -= ordered_matrix_data[i_row][j_col] * current_sol[j_col] # Uses already set basic or free vars

                       current_sol[pivot_col_idx] = rhs / pivot_val
                       
                  solutions.append([round(val, Matrix.round_precision) for val in current_sol])

        # Remove duplicates just in case generation method created them
        unique_solutions = []
        seen_solutions = set()
        for sol in solutions:
            sol_tuple = tuple(sol)
            if sol_tuple not in seen_solutions:
                unique_solutions.append(sol)
                seen_solutions.add(sol_tuple)
                
        return unique_solutions if unique_solutions else None


    
    def _remove_lz(vectors_list): #like
        if not vectors_list: return []

        matrix_data = [list(vec) for vec in vectors_list]
        original_indices = list(range(len(matrix_data)))

        rows = len(matrix_data)
        cols = len(matrix_data[0]) if rows > 0 else 0

        rank = 0
        pivot_row_indices = []

        for col in range(cols):
            if rank >= rows: break

            max_row_idx = rank
            for current_row_idx in range(rank + 1, rows):
                if abs(matrix_data[max_row_idx][col]) < abs(matrix_data[current_row_idx][col]):
                     max_row_idx = current_row_idx

            if abs(matrix_data[max_row_idx][col]) > 1e-10:
                matrix_data[rank], matrix_data[max_row_idx] = matrix_data[max_row_idx], matrix_data[rank]
                original_indices[rank], original_indices[max_row_idx] = original_indices[max_row_idx], original_indices[rank]

                pivot_value = matrix_data[rank][col]
                for i in range(rank + 1, rows):
                    factor = matrix_data[i][col] / pivot_value
                    for j in range(col, cols):
                        matrix_data[i][j] -= factor * matrix_data[rank][j]
                    if abs(matrix_data[i][col]) < 1e-10:
                        matrix_data[i][col] = 0.0

                pivot_row_indices.append(original_indices[rank])
                rank += 1

        return [vectors_list[i] for i in pivot_row_indices]

    def find_eigenvectors(self): #like
        if self.rows != self.cols: raise ValueError("Matrix must be square.")
        n = self.rows
        if n == 0: return [], []

        eigenvalues_result = self.find_eigenvalues()
        if not eigenvalues_result or not eigenvalues_result[0]:
             print("Could not find eigenvalues or no eigenvalues found.")
             return [], []

        eigenvalues, algebraic_multiplicities = eigenvalues_result
        all_eigenvectors = []
        geometric_multiplicities = []
        eigenpairs = [] # Store {'value': v, 'vectors': [list]}

        I = Matrix.identity(n)

        for eigenvalue, alg_mult in zip(eigenvalues, algebraic_multiplicities):
            lambda_I = I.scalar(eigenvalue)
            A_minus_lambda_I = self.sumM(lambda_I.scalar(-1.0))
            b_zero = [0.0] * n

            null_space_vectors = Matrix.solve(A_minus_lambda_I, b_zero, max_solves=n) # Request enough potential basis vectors

            if null_space_vectors is None:
                eigenvectors_for_lambda = []
            else:
                eigenvectors_for_lambda = [vec for vec in null_space_vectors if any(abs(x) > 1e-9 for x in vec)]

            independent_eigenvectors = Matrix._remove_lz(eigenvectors_for_lambda)
            
            # Normalize eigenvectors
            normalized_eigenvectors = []
            for vec in independent_eigenvectors:
                 norm = sum(x**2 for x in vec)**0.5
                 if norm > 1e-9:
                     normalized_eigenvectors.append([x / norm for x in vec])
                 else: # Keep zero vector if somehow generated (shouldn't be l.i.)
                      normalized_eigenvectors.append(vec)

            current_geom_mult = len(normalized_eigenvectors)
            geometric_multiplicities.append(current_geom_mult)
            all_eigenvectors.extend(normalized_eigenvectors)
            eigenpairs.append({'value': eigenvalue, 'vectors': normalized_eigenvectors, 'alg_mult': alg_mult, 'geom_mult': current_geom_mult})

            if current_geom_mult < alg_mult:
                 print(f"Warning: Geometric multiplicity ({current_geom_mult}) is less than algebraic multiplicity ({alg_mult}) for λ = {eigenvalue}. Matrix may not be diagonalizable.")
            elif current_geom_mult > alg_mult:
                 # This indicates a potential issue in calculation (g(λ) <= a(λ) always)
                 print(f"Warning: Calculated geometric multiplicity ({current_geom_mult}) > algebraic multiplicity ({alg_mult}) for λ = {eigenvalue}. Check calculations.")


        print("\nGeometric Multiplicities:")
        if not eigenpairs:
             print(" No eigenvectors found.")
        for i, pair in enumerate(eigenpairs, 1):
             print(f'g(λ_{i}) = {pair["geom_mult"]} for λ_{i} = {pair["value"]}')

        # Return list of all basis eigenvectors found, and list of geometric multiplicities corresponding to the order of eigenvalues from find_eigenvalues
        return all_eigenvectors, geometric_multiplicities

    
    
    def center_data(matrix_obj): #like
        if not isinstance(matrix_obj, Matrix): raise TypeError("Input must be a Matrix object")
        X_data = matrix_obj.data
        num_samples = matrix_obj.rows
        num_features = matrix_obj.cols
        if num_samples == 0: return Matrix([]), [0.0] * num_features

        means = [0.0] * num_features
        for j in range(num_features):
            col_sum = sum(X_data[i][j] for i in range(num_samples))
            means[j] = col_sum / num_samples

        centered_data = [[X_data[i][j] - means[j] for j in range(num_features)] for i in range(num_samples)]
        return Matrix(centered_data), means

    
    def covariance_matrix(matrix_obj): #like mb not like
         if not isinstance(matrix_obj, Matrix): raise TypeError("Input must be a Matrix object")
         X_centered = matrix_obj
         n_samples = X_centered.rows
         if n_samples <= 1:
              print("Warning: Covariance matrix is undefined or zero for <= 1 sample.")
              return Matrix([[0.0] * X_centered.cols for _ in range(X_centered.cols)])

         X_T = X_centered.tr()
         XTX = X_T.dot(X_centered)
         cov_matrix = XTX.scalar(1.0 / (n_samples - 1))
         return cov_matrix

    
    def explained_variance_ratio(eigenpairs, n_components): #like
        return sum(eig for eig, _ in eigenpairs[:n_components]) / sum(eig for eig, _ in eigenpairs)

    def auto_select_k(eigenvalues, threshold: float = 0.95):
        
        sorted_eigenvalues = sorted(eigenvalues, reverse=True)
        total_variance = sum(sorted_eigenvalues)
        cumulative_variance = 0.0
    
        for k, eigval in enumerate(sorted_eigenvalues, 1):
            cumulative_variance += eigval / total_variance
            if cumulative_variance >= threshold:
                return k
    
        return len(sorted_eigenvalues)

    
    def PCA(input_matrix, n_components=None, threshold=0.95):

        # Проверка входных данных
        if not isinstance(input_matrix, Matrix):
            raise TypeError("Input must be a Matrix object")
        if input_matrix.rows == 0 or input_matrix.cols == 0:
            return Matrix([]), Matrix([]), [], 0.0
    
        # 1. Центрирование данных
        X_centered, means = Matrix.center_data(input_matrix)
    
        # 2. Вычисление ковариационной матрицы
        cov_mat = Matrix.covariance_matrix(X_centered)
    
        # 3. Нахождение собственных значений и векторов (улучшенный метод)
        try:
            eigenvalues, eigenvectors = cov_mat.find_eigenvalues_and_vectors()
            if not eigenvalues or not eigenvectors:
                raise ValueError("No eigenvalues/vectors found")
        except Exception as e:
            print(f"PCA failed in eigenvalue decomposition: {str(e)}")
            return Matrix([]), Matrix([]), means, 0.0
    
        # 4. Сортировка по убыванию собственных значений
        eigenpairs = sorted(zip(eigenvalues, eigenvectors), 
                       key=lambda x: -x[0])
    
        # 5. Автоматический выбор числа компонент
        if n_components is None:
            n_components = Matrix.auto_select_k(eigenvalues, threshold)
        else:
            n_components = min(n_components, len(eigenvectors))
    
        # 6. Выбор главных компонент
        components = [vec for eig, vec in eigenpairs[:n_components]]
        if not components:
            print("Warning: No principal components selected")
            return Matrix([]), Matrix([]), means, 0.0
    
        # 7. Формирование матрицы проекции
        W = Matrix(components)  # Транспонируем для правильной размерности
    
        # 8. Проекция данных
        X_proj = X_centered.dot(W)
    
        # 9. Вычисление объясненной дисперсии
        explained_variance = Matrix.explained_variance_ratio(eigenpairs, n_components)
    
        return X_proj, W, means, explained_variance
    
    
    def inverse_PCA(X_proj, components, means):
         if not isinstance(X_proj, Matrix) or not isinstance(components, Matrix):
             raise TypeError("Inputs X_proj and components must be Matrix objects")

         W = components # (k x d), eigenvectors as rows
         if X_proj.cols != W.rows:
              raise ValueError(f"Projected data columns ({X_proj.cols}) must match number of components ({W.rows})")
         if W.cols != len(means):
              raise ValueError(f"Component feature dimension ({W.cols}) must match number of means ({len(means)})")


         X_rec_centered = X_proj.dot(W.tr()) # (n x k) dot (k x d) -> (n x d)

         reconstructed_data = [[X_rec_centered.data[i][j] + means[j] for j in range(len(means))]
                               for i in range(X_rec_centered.rows)]

         return Matrix(reconstructed_data)
    
    
    def reconstruction_error(X_orig, X_recon):
        if X_orig.rows != X_recon.rows or X_orig.cols != X_recon.cols:
            raise ValueError("Размерности исходной и восстановленной матриц должны совпадать")
    
        total_squared_error = 0.0
        for i in range(X_orig.rows):
            for j in range(X_orig.cols):
                error = X_orig.data[i][j] - X_recon.data[i][j]
                total_squared_error += error ** 2
    
        mse = total_squared_error / (X_orig.rows * X_orig.cols)
        return mse
    
    def plot_pca_projection(X_pca):
        if X_pca.cols != 2:
            raise ValueError("Projection matrix must have 2 columns")
    
        x = [row[0] for row in X_pca.data]
        y = [row[1] for row in X_pca.data]
    
        fig, ax = plt.subplots()
        ax.scatter(x, y)
        ax.set_xlabel('First Principal Component')
        ax.set_ylabel('Second Principal Component')
        ax.set_title('PCA Projection')
    
        return fig
    
    def handle_missing_values(X):
        filled_data = [row.copy() for row in X.data]
        n_rows, n_cols = X.rows, X.cols
    
        for j in range(n_cols):
            # Собираем все не-NaN значения в столбце
            column_values = []
            for i in range(n_rows):
                val = filled_data[i][j]
                if val is not None and not (isinstance(val, float) and math.isnan(val)):
                    column_values.append(val)
        
            if not column_values:
                column_mean = 0.0  # если все значения NaN, заменяем на 0
            else:
                column_mean = sum(column_values) / len(column_values)
        
            # Заменяем NaN на среднее
            for i in range(n_rows):
                val = filled_data[i][j]
                if val is None or (isinstance(val, float) and math.isnan(val)):
                    filled_data[i][j] = column_mean
    
        return Matrix(filled_data)
    def apply_pca_to_dataset(dataset_name, k: int = None, threshold: float = 0.95):
        # Загрузка данных (пример для встроенных датасетов)
        if dataset_name == 'iris':
            from sklearn.datasets import load_iris
            data = load_iris()
            X = Matrix(data['data'].tolist())
            y = data['target']
        elif dataset_name == 'wine':
            from sklearn.datasets import load_wine
            data = load_wine()
            X = Matrix(data['data'].tolist())
            y = data['target']
        else:
            # Для CSV (предполагается, что последний столбец — целевая переменная)
            import csv
            with open(dataset_name, 'r') as f:
                reader = csv.reader(f)
                data = [list(map(float, row)) for row in reader]
            X = Matrix([row[:-1] for row in data])
            y = [row[-1] for row in data]
    
        # Выполнение PCA
        X_proj, components, means, ratio = Matrix.PCA(X, k)
    
        # Восстановление данных для оценки ошибки
        X_recon = Matrix.inverse_PCA(X_proj, components, means)
        mse = Matrix.reconstruction_error(X, X_recon)
        return {
            'X_proj': X_proj,
            'explained_variance': ratio,
            'mse': mse
        }

    def add_noise_and_compare(X, noise_level: float = 0.1):
        # 1. Центрируем данные и вычисляем стандартные отклонения
        X_centered, means = Matrix.center_data(X)
        std_devs = [math.sqrt(sum(X_centered.data[i][j]**2 for i in range(X.rows)) / (X.rows - 1)) for j in range(X.cols)]
    
    
        # 2. Генерируем зашумленные данные
        noisy_data = [
            [
                X.data[i][j] + random.gauss(0, std_devs[j] * noise_level)
                for j in range(X.cols)
            ]
            for i in range(X.rows)
        ]
        X_noisy = Matrix(noisy_data)
    
        # 3. Выполняем PCA для исходных данных
        X_proj_orig, components_orig, means_orig, ratio_orig = Matrix.PCA(X, n_components=X.cols)
        X_recon_orig = Matrix.inverse_PCA(X_proj_orig, components_orig, means)
        mse_orig = Matrix.reconstruction_error(X, X_recon_orig)
    
        # 4. Выполняем PCA для зашумленных данных
        X_proj_noisy, components_noisy, means_noisy, ratio_noisy = Matrix.PCA(X_noisy, n_components=X.cols)
    
        # 5. Сравнение собственных значений
        cov_orig = Matrix.covariance_matrix(X_centered)
        eigenvalues_orig, eigenvaluesvectors_orig = cov_orig.find_eigenvalues_and_vectors()
    
        X_noisy_centered, means_noisy = Matrix.center_data(X_noisy)
        cov_noisy = Matrix.covariance_matrix(X_noisy_centered)
        eigenvalues_noisy, eigenvaluesvectors_noisy = cov_noisy.find_eigenvalues_and_vectors()
    
        return {
            'eigenvalues_orig': eigenvalues_orig,
            'eigenvalues_noisy': eigenvalues_noisy,
            'explained_var_orig': ratio_orig,
            'explained_var_noisy': ratio_noisy,
            'mse_reconstruction': mse_orig
        }

In [5]:
M_data = [
[1, -2, 0, 0],
[4, 7, 0, 0],
[-6, -5, 1, -1],
[8, 6, 4, 5]
]

def I(n):
    m_ = []
    for i in range(n):
        m_.append([0 for j in range(n)])
        m_[-1][i] = 1
    return m_

M = Matrix(M_data)
X_centered, _ = Matrix.center_data(M)
cov_mat = Matrix.covariance_matrix(X_centered)


In [6]:
print("Original Matrix M:")
for row in M.data: print(row)

try:
    # Perform PCA
    # n_components can be set, e.g., n_components=2
    n = 4
    X_pca, components, means, ratio = Matrix.PCA(M, n_components=n)
    if n == 2:
        Matrix.plot_pca_projection(X_pca)
    print("\nPCA Projected Data (X_pca):")
    if X_pca.data:
            for row in X_pca.data: print([round(x, 5) for x in row])
    else: print(" Empty")

    print("\nPrincipal Components (Eigenvectors as rows):")
    if components.data:
            for row in components.data: print([round(x, 5) for x in row])
    else: print(" Empty")

    print("\nMeans:")
    print([round(x, 5) for x in means])

    # Reconstruct data
    M_reconstructed = Matrix.inverse_PCA(X_pca, components, means)

    print("\nReconstructed Matrix:")
    if M_reconstructed.data:
            for row in M_reconstructed.data: print([round(x, 5) for x in row])
    else: print(" Empty")
    print("1 ошибка и ты ошибся", Matrix.reconstruction_error(M, M_reconstructed))
    
    a = Matrix.add_noise_and_compare(M)
    print(a)
    print(ratio)

except Exception as e:
    import traceback
    print(f"\nAn error occurred during PCA processing: {e}")
    print(traceback.format_exc())

Original Matrix M:
[1, -2, 0, 0]
[4, 7, 0, 0]
[-6, -5, 1, -1]
[8, 6, 4, 5]

PCA Projected Data (X_pca):
[0.90569, -2.98159, 0.39137, 2.3477]
[0.80346, 4.05714, -3.91581, -2.33141]
[-4.05812, -8.80564, 1.03501, 3.36098]
[2.34896, 7.73008, 2.48943, -3.37727]

Principal Components (Eigenvectors as rows):
[0.68719, 0.67238, 0.114, 0.25036]
[-0.24042, 0.55795, -0.51657, -0.60335]
[-0.64639, 0.48042, 0.54357, 0.23645]
[0.22837, -0.07606, 0.65168, -0.71929]

Means:
[1.75, 1.5, 1.25, 1.0]

Reconstructed Matrix:
[1.0, -2.0, -0.0, 0.0]
[4.0, 7.0, -0.0, -0.0]
[-6.0, -5.0, 1.0, -1.0]
[8.0, 6.0, 4.0, 5.0]
1 ошибка и ты ошибся 3.386623192149685e-22
{'eigenvalues_orig': [71.10261199719706, 7.568870657087935, 2.1620684711764144, -0.0005511254614031583], 'eigenvalues_noisy': [67.40150331637976, 10.036306253370496, 2.4720787259382355, 0.00011170431152687926], 'explained_var_orig': 1.0, 'explained_var_noisy': 1.0, 'mse_reconstruction': 3.386623192149685e-22}
1.0
